<a href="https://colab.research.google.com/github/RidhaBouhiaoui/Trading/blob/main/Tbotbasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install python-binance
!pip install pandas


In [13]:
from binance import Client
import pandas as pd


In [14]:
api_key = 'vBH5GvjHWAxF0jnX0ps8NXg624kpjWqdvmHyyutAFUbG3F878FaiPOpN6FHoamYW'
api_secret = 'MBvDArXtId3b8UaIQk1F7Siy7ZlmmQCTmYVozjOGyIwSgGK6nHztUnLzjaSIZWjw'

In [15]:
client = Client(api_key, api_secret)

In [16]:
#datastream via websocket - will not be covered in this code. 

In [17]:
pd.DataFrame(client.get_historical_klines('BTCUSDT', '1m', '30 m ago UTC'))


,0,1,2,3,4,5,6,7,8,9,10,11
0,1637526480000,59405.21000000,59430.57000000,59398.60000000,59414.84000000,12.45652000,1637526539999,740092.49420510,612,5.19358000,308576.45156950,0
1,1637526540000,59414.85000000,59432.30000000,59407.65000000,59432.30000000,13.54232000,1637526599999,804677.96724260,677,8.04230000,477868.07910050,0
2,1637526600000,59432.29000000,59449.10000000,59390.85000000,59390.86000000,14.10884000,1637526659999,838503.44125090,828,4.52455000,268896.69746670,0
3,1637526660000,59390.86000000,59410.00000000,59363.69000000,59394.79000000,14.48887000,1637526719999,860369.48530010,786,3.29139000,195441.13839930,0
4,1637526720000,59394.79000000,59398.63000000,59357.75000000,59371.30000000,8.22096000,1637526779999,488178.10307610,664,2.99081000,177597.11649400,0
5,1637526780000,59371.29000000,59384.55000000,59353.01000000,59378.01000000,12.63618000,1637526839999,750198.84345720,754,4.81820000,286045.93423660,0
6,1637526840000,59378.91000000,59394.70000000,59378.00000000,59381.60000000,8.92800000,1637526899999,530203.40395330,707,4.44139000,263756.80058910,0
7,1637526900000,59381.60000000,59432.42000000,59381.59000000,59428.75000000,18.00020000,1637526959999,1069403.81641680,830,10.52830000,625509.79785310,0
8,1637526960000,59428.74000000,59485.06000000,59428.74000000,59470.63000000,23.24136000,1637527019999,1381969.78623150,988,9.70646000,577163.23147440,0
9,1637527020000,59470.63000000,59496.40000000,59464.66000000,59488.19000000,23.15675000,1637527079999,1377382.49997140,804,15.72454000,935309.12515040,0


In [18]:
#data manipulation
def getminutedata(symbol, interval, lookback):
   frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback+' min ago UTC'))
   frame = frame.iloc[:,:6]
   frame.columns = ['Time', 'Open', 'High', 'Low','Close', 'Volume']
   frame = frame.set_index('Time')
   frame.index = pd.to_datetime(frame.index, unit='ms')
   frame = frame.astype(float)
   return frame

In [19]:
############# how to to obtain data
##        test = getminutedata('BTCUSDT', '1h', '2400')  ##
############# this is how you plot a graph
##        test.Open.plot()           ##

In [36]:
############ MY AIM IS TO BUILD A STRATEGY THAT CONSIDERS THE FOLLOWING
## RSI UNDER 30/ OVER 70 LINE ##
## EMAS IN ORDER AND/OR EMA CROSS ##
## BTC RSI ENTERED SUB30 OR ABOVE70 THEN CLOSES IN RANGE 50-70 OR 3-50 ##
## 5:1 RR, SL TO BE ONCE SECURE 3RR EG 1% RISK FOR 5%GAIN ##
## MAC D IN ORDER TOO, THIS MAY BE OVERCOMPLICATING ##
##  ##

In [ ]:
#buy if asset fell by more then 0.2%
#sell if asset rises more then .15% or falls further .15%



In [29]:
def strategytest(symbol, qty, tf, tb, entered = False):
    df = getminutedata(symbol, tf, tb)
    #looking at asset performance
    cumulret = (df.Open.pct_change()+1).cumprod() - 1
    if not entered:
     ## These are the conditions for entry. ##    
        if cumulret[-1] < -0.002:
           order = client.create_order(symbol = symbol,
                                       side = "BUY", type = 'MARKET',
                                       quantity=qty)
           print(order)
           entered = True
        else:
             print('No Trade has been executed')
    if entered:
       while True:
            df = getminutedata(symbol, tf, tb)
            sincebuy = df.loc[df.index > pd.to_datetime(
            order['transactTime'], unit='ms')]
            if len(sincebuy) > 0 :
               sincebuyret = (sincebuy.Open.pct_change()+1).cumprod() - 1
        ## This is the conditions for exiting the position. ##
               if sincebuyret[-1] > 0.0015 or sincebuyret[-1] < -0.0015:
                  order = client.create_order(symbol = symbol,
                                       side = "SELL", type = 'MARKET',
                                       quantity=qty)
                  print(order)
                  break


In [37]:
strategytest('BTCUSDT', 0.001, '1m', '30m')

No Trade has been executed
